In [7]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import tree
from os import system
import seaborn as sns
import pandas as pd
import numpy as np
import warnings